## 1) Import libraries
---

In [18]:
import plotly.figure_factory as ff
import plotly.express as px
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import   StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

from dotenv import load_dotenv
import os

import boto3
from io import StringIO

## 2) Import files
---

In [2]:
load_dotenv()

aws_access_key = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")

# Initialiser un client S3
s3 = boto3.client(
    "s3",
    region_name="eu-west-3",
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key
)

# Télécharger le fichier
bucket_name = "housing-prices-aurelien"
key = "real_estate_dataset.csv"

response = s3.get_object(Bucket=bucket_name, Key=key)
csv_content = response["Body"].read().decode("utf-8")

# Charger dans un DataFrame
data = pd.read_csv(StringIO(csv_content))

# Aperçu
data.head()


,ID,Square_Feet,Num_Bedrooms,Num_Bathrooms,Num_Floors,Year_Built,Has_Garden,Has_Pool,Garage_Size,Location_Score,Distance_to_Center,Price
0,1,143.635030,1,3,3,1967,1,1,48,8.297631,5.935734,602134.816747
1,2,287.678577,1,2,1,1949,0,1,37,6.061466,10.827392,591425.135386
2,3,232.998485,1,3,2,1923,1,0,14,2.911442,6.904599,464478.696880
3,4,199.664621,5,2,2,1918,0,0,17,2.070949,8.284019,583105.655996
4,5,89.004660,4,3,3,1999,1,0,34,1.523278,14.648277,619879.142523


## 3) EDA
---

In [3]:
# Basic stats
print(f"Number of rows : {data.shape[0]}")
print(f"Number of columns : {data.shape[1]}")

print("Basics statistics: ")
display(data.describe(include='all'))

print("Percentage of missing values: ")
print(data.isna() .sum() / data.shape[0])

Number of rows : 500
Number of columns : 12
Basics statistics: 


,ID,Square_Feet,Num_Bedrooms,Num_Bathrooms,Num_Floors,Year_Built,Has_Garden,Has_Pool,Garage_Size,Location_Score,Distance_to_Center,Price
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,250.500000,174.640428,2.958000,1.976000,1.964000,1957.604000,0.536000,0.492000,30.174000,5.164410,10.469641,582209.629529
std,144.481833,74.672102,1.440968,0.820225,0.802491,35.491781,0.499202,0.500437,11.582575,2.853489,5.588197,122273.390345
min,1.000000,51.265396,1.000000,1.000000,1.000000,1900.000000,0.000000,0.000000,10.000000,0.004428,0.062818,276892.470136
25%,125.750000,110.319923,2.000000,1.000000,1.000000,1926.000000,0.000000,0.000000,20.000000,2.760650,6.066754,503080.344140
50%,250.500000,178.290937,3.000000,2.000000,2.000000,1959.000000,1.000000,0.000000,30.000000,5.206518,10.886066,574724.113347
75%,375.250000,239.031220,4.000000,3.000000,3.000000,1988.000000,1.000000,1.000000,41.000000,7.732933,15.072590,665942.301274
max,500.000000,298.241199,5.000000,3.000000,3.000000,2022.000000,1.000000,1.000000,49.000000,9.995439,19.927966,960678.274291


Percentage of missing values: 
ID                    0.0
Square_Feet           0.0
Num_Bedrooms          0.0
Num_Bathrooms         0.0
Num_Floors            0.0
Year_Built            0.0
Has_Garden            0.0
Has_Pool              0.0
Garage_Size           0.0
Location_Score        0.0
Distance_to_Center    0.0
Price                 0.0
dtype: float64


In [4]:
# Distribution of each numeric variable
num_features = ['Square_Feet','Garage_Size','Location_Score','Distance_to_Center', 'Price']
for i in range(len(num_features)):
    fig = px.histogram(data[num_features[i]])
    fig.show()

In [5]:
#Barplot of each qualitative variable

qual_features = ['Num_Bedrooms','Num_Bathrooms','Num_Floors','Year_Built', 'Has_Garden', 'Has_Pool']

for i in range(len(qual_features)):
    df = pd.DataFrame(data[qual_features[i]].value_counts()).reset_index()
    df.columns = [qual_features[i], 'Count']
    fig = px.bar(df, x=qual_features[i], y='Count', title=qual_features[i])
    fig.show()

In [6]:
# Correlation matrix
corr_matrix = data.corr().round(2)

fig = ff.create_annotated_heatmap(corr_matrix.values,
                                  x = corr_matrix.columns.tolist(),
                                  y = corr_matrix.index.tolist())


fig.show()

## 4) Preprocessing
---

In [7]:
#Drop lines containing outliers :
def drop_outlier(dataset, col):
    upper_outlier = dataset[col].mean() + 3 * dataset[col].std()
    lower_outlier = dataset[col].mean() - 3 * dataset[col].std()
    outlier_condition = (dataset[col] > upper_outlier) | (dataset[col] < lower_outlier)
    dataset = dataset[~outlier_condition]

    return dataset

Columns_to_clean = ['Price']
for col in Columns_to_clean:
    df = drop_outlier(data,col)

In [8]:
#Separate target from other explanatory variable :
target_variable = "Price"
X = df.drop([target_variable,'ID'] , axis = 1)
y = df[target_variable]

my_features_list = X.columns.tolist() #for later


#Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


#Numeric/categorical columns
categorical_features = []
numeric_features = [feature for feature in my_features_list if feature not in categorical_features]


#Transformer
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numeric_features)
        ])

#Preprocess
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [20]:
print("NaN dans X_train :", np.isnan(X_train).sum())
print("Inf dans X_train :", np.isinf(X_train).sum())

NaN dans X_train : 0
Inf dans X_train : 0


## 5) Baseline model (linear regression)
---

In [9]:
scoring_df = pd.DataFrame()

def log_model_score(df, model_name, R2_train, R2_test):
    rows = pd.DataFrame({
        'model_name': [model_name, model_name],
        'series': ['Train', 'Test'],
        'r2_score': [R2_train, R2_test]
    })
    return pd.concat([df, rows], ignore_index=True)


In [10]:
# Train model
regressor = LinearRegression()
regressor.fit(X_train, y_train)


# Predictions
y_train_pred = regressor.predict(X_train)
y_test_pred = regressor.predict(X_test)

# Print and save R2 scores
R2_train = r2_score(y_train, y_train_pred)
R2_test = r2_score(y_test, y_test_pred)
print("R2 score on training set : ", R2_train)
print("R2 score on test set : ", R2_test)

model_name = "Linear Regression"
scoring_df = log_model_score(scoring_df,model_name,R2_train, R2_test)


R2 score on training set :  0.9762006396979073
R2 score on test set :  0.9713328388699407


/opt/anaconda3/envs/lead-projet-final-env/lib/python3.10/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning:

divide by zero encountered in matmul

/opt/anaconda3/envs/lead-projet-final-env/lib/python3.10/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning:

overflow encountered in matmul

/opt/anaconda3/envs/lead-projet-final-env/lib/python3.10/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning:

invalid value encountered in matmul



In [11]:
# Create a pandas DataFrame
column_names = []
for name, pipeline, features_list in preprocessor.transformers_:
    if name == 'num':
        features = features_list
    else:
        features = pipeline.named_steps['encoder'].get_feature_names_out()
    column_names.extend(features)
        
coefs = pd.DataFrame(index = column_names, data = regressor.coef_.transpose(), columns=["coefficients"])
feature_importance = abs(coefs).sort_values(by = 'coefficients')


# Plot coefficients
fig = px.bar(feature_importance, orientation = 'h')
fig.update_layout(showlegend = False, 
                  margin = {'l': 120} 
                 )
fig.show()

## 6) Ridge & Lasso
---

In [12]:
# Ridge
regressor = Ridge()
regressor.fit(X_train, y_train)

# Print and save R2 scores
R2_train = regressor.score(X_train, y_train)
R2_test = regressor.score(X_test, y_test)
print("R2 score on training set : ", R2_train)
print("R2 score on test set : ", R2_test)

model_name = "Ridge"
scoring_df = log_model_score(scoring_df,model_name,R2_train, R2_test)


R2 score on training set :  0.9761931103461313
R2 score on test set :  0.9712738486387501


/opt/anaconda3/envs/lead-projet-final-env/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

divide by zero encountered in matmul

/opt/anaconda3/envs/lead-projet-final-env/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

overflow encountered in matmul

/opt/anaconda3/envs/lead-projet-final-env/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

invalid value encountered in matmul

/opt/anaconda3/envs/lead-projet-final-env/lib/python3.10/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning:

divide by zero encountered in matmul

/opt/anaconda3/envs/lead-projet-final-env/lib/python3.10/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning:

overflow encountered in matmul

/opt/anaconda3/envs/lead-projet-final-env/lib/python3.10/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning:

invalid value encountered in matmul



In [13]:
regressor = Lasso()
regressor.fit(X_train, y_train)

# Print and save R2 scores
R2_train = regressor.score(X_train, y_train)
R2_test = regressor.score(X_test, y_test)
print("R2 score on training set : ", R2_train)
print("R2 score on test set : ", R2_test)

model_name = "Lasso"
scoring_df = log_model_score(scoring_df,model_name,R2_train, R2_test)

R2 score on training set :  0.9762006388681125
R2 score on test set :  0.9713317552439752


/opt/anaconda3/envs/lead-projet-final-env/lib/python3.10/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning:

divide by zero encountered in matmul

/opt/anaconda3/envs/lead-projet-final-env/lib/python3.10/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning:

overflow encountered in matmul

/opt/anaconda3/envs/lead-projet-final-env/lib/python3.10/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning:

invalid value encountered in matmul



**==> Standard Ridge and Lasso have the sames resultats that the linear Regression in this case**

## 7) GridSearch
---

In [14]:
# Ridge with gridsearch

regressor = Ridge()
params = {
    'alpha': [0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100] # 0 corresponds to no regularization
}
gridsearch_ridge = GridSearchCV(regressor, param_grid=params, cv=3) # cv : the number of folds to be used for CV
gridsearch_ridge.fit(X_train, y_train)

print("Best hyperparameters : ", gridsearch_ridge.best_params_)
print("Best R2 score : ", gridsearch_ridge.best_score_)

# Print and save R2 scores
R2_train = gridsearch_ridge.score(X_train, y_train)
R2_test = gridsearch_ridge.score(X_test, y_test)
print("R2 score on training set : ", R2_train)
print("R2 score on test set : ", R2_test)

model_name = "Ridge with gridsearch"
scoring_df = log_model_score(scoring_df,model_name,R2_train, R2_test)

Best hyperparameters :  {'alpha': 0.01}
Best R2 score :  0.9737930165057401
R2 score on training set :  0.9762006389407066
R2 score on test set :  0.9713323217447846


/opt/anaconda3/envs/lead-projet-final-env/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

divide by zero encountered in matmul

/opt/anaconda3/envs/lead-projet-final-env/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

overflow encountered in matmul

/opt/anaconda3/envs/lead-projet-final-env/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning:

invalid value encountered in matmul

/opt/anaconda3/envs/lead-projet-final-env/lib/python3.10/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning:

divide by zero encountered in matmul

/opt/anaconda3/envs/lead-projet-final-env/lib/python3.10/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning:

overflow encountered in matmul

/opt/anaconda3/envs/lead-projet-final-env/lib/python3.10/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning:

invalid value encountered in matmul

/opt/anaconda3/envs/lead-projet-final-env/lib/python3.10/site-p

In [15]:
# Lasso with gridsearch

regressor = Lasso()
params = {
    'alpha': [100, 500, 1000, 1500, 2000] # 0 corresponds to no regularization
}
gridsearch_lasso = GridSearchCV(regressor, param_grid=params, cv=3) # cv : the number of folds to be used for CV
gridsearch_lasso.fit(X_train, y_train)

print("Best hyperparameters : ", gridsearch_lasso.best_params_)
print("Best R2 score : ", gridsearch_lasso.best_score_)

# Print and save R2 scores
R2_train = gridsearch_lasso.score(X_train, y_train)
R2_test = gridsearch_lasso.score(X_test, y_test)
print("R2 score on training set : ", R2_train)
print("R2 score on test set : ", R2_test)

model_name = "Lasso with gridsearch"
scoring_df = log_model_score(scoring_df,model_name,R2_train, R2_test)

Best hyperparameters :  {'alpha': 100}
Best R2 score :  0.9737728453677504
R2 score on training set :  0.9761924093378729
R2 score on test set :  0.97121537977537


/opt/anaconda3/envs/lead-projet-final-env/lib/python3.10/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning:

divide by zero encountered in matmul

/opt/anaconda3/envs/lead-projet-final-env/lib/python3.10/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning:

overflow encountered in matmul

/opt/anaconda3/envs/lead-projet-final-env/lib/python3.10/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning:

invalid value encountered in matmul

/opt/anaconda3/envs/lead-projet-final-env/lib/python3.10/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning:

divide by zero encountered in matmul

/opt/anaconda3/envs/lead-projet-final-env/lib/python3.10/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning:

overflow encountered in matmul

/opt/anaconda3/envs/lead-projet-final-env/lib/python3.10/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning:

invalid value encountered in matmul

/opt/anaconda3/envs/lead-projet-final-env/lib/py

In [16]:
#show the feature coefficients from Ridge and Lasso
data_dict = {
    'Feature': column_names,
    'Best_Ridge': gridsearch_ridge.best_estimator_.coef_,
    'Best_Lasso': gridsearch_lasso.best_estimator_.coef_,
            }

coefficients_ridge_lasso = pd.DataFrame(data=data_dict)

fig = px.line(coefficients_ridge_lasso, x = 'Feature', y = ['Best_Ridge', 'Best_Lasso'])
fig.show()

# Conclusion
---

In [17]:

fig = px.line(scoring_df, x='model_name', y='r2_score', color='series',
              markers=True, title="Scores R² pour chaque modèle")
fig.show()


**==> We will keep the baseline regression for our model. Lasso, Ridge and gridsearch have the same results**